# Firenze card logs need deduplication
First, import relevant libraries:

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from pylab import *

import igraph as ig

In [2]:
import sys
sys.path.append('../../src/')
from utils.database import dbutils

conn = dbutils.connect()
cursor = conn.cursor()

Then, load the data (takes a few moments):

In [3]:
df = pd.read_sql('select * from optourism.firenze_card_logs', con=conn)
df.head()

,user_id,museum_name,entry_time,adults_first_use,adults_reuse,total_adults,minors,museum_id
0,2047835,Battistero di San Giovanni,2016-08-01 08:37:00,0,1,1,0,2
1,2047834,Battistero di San Giovanni,2016-08-01 08:37:00,0,1,1,0,2
2,2067906,Battistero di San Giovanni,2016-08-01 08:39:00,0,1,1,0,2
3,2067905,Battistero di San Giovanni,2016-08-01 08:40:00,0,1,1,0,2
4,2068678,Battistero di San Giovanni,2016-08-01 08:42:00,0,1,1,0,2


In [4]:
def frequency(dataframe,columnname):
    out = dataframe[columnname].value_counts().to_frame()
    out.columns = ['frequency']
    out.index.name = columnname
    out.reset_index(inplace=True)
    out = out.sort_values(columnname)
    out['cumulative'] = out['frequency'].cumsum()/out['frequency'].sum()
    out['ccdf'] = 1 - out['cumulative']
    return out

In [6]:
(df['adults_first_use'] + df['adults_reuse'] != df['total_adults']).sum() # Check to make sure the columns add up

0

In [7]:
(df['total_adults'] > 1).sum() # Check to make sure there is never more than 1 adult per card, acc

0

In [9]:
fr1 = frequency(df,'minors')
fr1.head() # Only 1 child max per card, which is about 10% of the cases

,minors,frequency,cumulative,ccdf
0,0,360541,0.907898,0.092102
1,1,36575,1.000000,0.000000


In [47]:
# Now, do some people visit the same museum more than once?
fr2 = frequency(df.groupby(['museum_name','user_id'])['total_adults'].sum().to_frame(),'total_adults')
fr2.head(20) # Only 19 people visited a place more than once. 

,total_adults,frequency,cumulative,ccdf
0,1,360503,0.999947,0.000053
1,2,19,1.000000,0.000000


In [52]:
df1 = df.groupby(['museum_name','user_id'])['minors'].sum().to_frame()
fr3 = frequency(df1,'minors')
fr3

,minors,frequency,cumulative,ccdf
0,0,333474,0.924975,0.075025
1,1,19471,0.978983,0.021017
2,2,5913,0.995384,0.004616
3,3,1512,0.999578,0.000422
4,4,111,0.999886,0.000114
5,5,19,0.999939,0.000061
6,6,7,0.999958,0.000042
10,7,2,0.999964,0.000036
7,9,6,0.999981,0.000019
9,11,2,0.999986,0.000014


In [53]:
df1[df1['minors']>10]

minors
museum_name                                        user_id        
Battistero di San Giovanni                         2058028      14
Galleria degli Uffizi                              2068648      19
Galleria dell'Accademia di Firenze                 2078823      11
Museo Nazionale del Bargello                       2049800      13
                                                   2069827      12
Museo di Santa Maria Novella                       2075251      13
Palazzo Pitti 2 Ð Giardino di Boboli, Museo deg... 2080579      11

In [24]:
# Check to see what the case of 19 looks like.
df[(df['user_id']==2068648) & (df['museum_name']=='Galleria degli Uffizi')]

,user_id,museum_name,entry_time,adults_first_use,adults_reuse,total_adults,minors,museum_id
21430,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21431,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21432,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21433,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21434,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21435,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21436,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21437,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21439,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9
21440,2068648,Galleria degli Uffizi,2016-08-06 16:29:00,0,0,0,1,9


In [44]:
df2 = df.groupby(['user_id','museum_name','entry_time']).sum()
df2[(df2['total_adults']>1)|(df2['minors']>1)].head(50)

adults_first_use  \
user_id museum_name                                        entry_time                              
2017468 Battistero di San Giovanni                         2016-06-16 10:46:00                 1   
        Galleria degli Uffizi                              2016-06-18 10:45:00                 0   
        Galleria dell'Accademia di Firenze                 2016-06-16 12:56:00                 0   
        Museo Galileo                                      2016-06-18 15:56:00                 0   
        Museo di Palazzo Vecchio                           2016-06-18 13:17:00                 0   
        Palazzo Medici Riccardi                            2016-06-16 12:00:00                 0   
        Palazzo Pitti 2 Ð Giardino di Boboli, Museo deg... 2016-06-17 12:34:00                 0   
        Torre di Palazzo Vecchio                           2016-06-18 14:58:00                 0   
2017470 Battistero di San Giovanni                         2016-06-16 12:07:00                 0   
        Galleria degli Uffizi                              2016-06-16 09:41:00                 1   
        Galleria dell'Accademia di Firenze                 2016-06-16 14:19:00                 0   
2017487 Battistero di San Giovanni                         2016-06-15 13:13:00                 0   
        Galleria degli Uffizi                              2016-06-15 11:26:00                 0   
        Galleria dell'Accademia di Firenze                 2016-06-15 16:10:00                 0   
        Museo Galileo                                      2016-06-14 12:14:00                 1   
        Museo Nazionale del Bargello                       2016-06-14 14:04:00                 0   
        Museo di Palazzo Vecchio                           2016-06-14 21:35:00                 0   
2017818 Museo Nazionale del Bargello                       2016-06-22 10:24:00                 1   
        Palazzo Pitti 2 Ð Giardino di Boboli, Museo deg... 2016-06-24 10:58:00                 0   
2017844 Museo di San Marco                                 2016-06-20 11:13:00                 0   
2027552 Basilica San Lorenzo                               2016-06-02 16:32:00                 0   
        Basilica di Santa Croce                            2016-06-04 09:29:00                 0   
        Battistero di San Giovanni                         2016-06-02 13:08:00                 0   
        Casa Buonarroti                                    2016-06-04 10:16:00                 0   
        Galleria degli Uffizi                              2016-06-02 09:15:00                 1   
        Galleria dell'Accademia di Firenze                 2016-06-02 15:46:00                 0   
        La Specola                                         2016-06-03 12:42:00                 0   
        Museo Casa Dante                                   2016-06-04 10:47:00                 0   
        Museo Galileo                                      2016-06-03 16:58:00                 0   
        Museo Novecento                                    2016-06-03 18:30:00                 0   
        Museo di Palazzo Vecchio                           2016-06-03 15:23:00                 0   
        Museo di Santa Maria Novella                       2016-06-02 17:37:00                 0   
        Palazzo Pitti 2 Ð Giardino di Boboli, Museo deg... 2016-06-03 09:37:00                 0   
        Palazzo Strozzi                                    2016-06-03 13:21:00                 0   
        Torre di Palazzo Vecchio                           2016-06-03 16:14:00                 0   
2027571 Battistero di San Giovanni                         2016-06-03 11:18:00                 0   
        Galleria degli Uffizi                              2016-06-02 11:34:00                 0   
        Galleria dell'Accademia di Firenze                 2016-06-02 10:00:00                 1   
        Museo Galileo                                      2016-06-02 14:32:00  

In [43]:
df[df['user_id']==2017844]

,user_id,museum_name,entry_time,adults_first_use,adults_reuse,total_adults,minors,museum_id
154378,2017844,Museo Nazionale del Bargello,2016-06-18 12:03:00,1,0,1,0,31
154551,2017844,Museo di Palazzo Vecchio,2016-06-18 12:36:00,0,1,1,0,22
154762,2017844,Torre di Palazzo Vecchio,2016-06-18 13:08:00,0,1,1,0,40
154920,2017844,Museo Galileo,2016-06-18 13:46:00,0,1,1,0,28
155069,2017844,Galleria degli Uffizi,2016-06-18 14:25:00,0,1,1,0,9
156874,2017844,Cappelle Medicee,2016-06-19 11:52:00,0,1,1,0,5
159026,2017844,Museo di San Marco,2016-06-20 11:13:00,0,0,0,1,24
159027,2017844,Museo di San Marco,2016-06-20 11:13:00,0,0,0,1,24
159028,2017844,Museo di San Marco,2016-06-20 11:13:00,0,0,0,1,24
159029,2017844,Museo di San Marco,2016-06-20 11:13:00,0,0,0,1,24
